In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
data = pd.read_csv('cleaned_file.csv')

In [2]:
# df.dtypes

In [3]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

# Initialize known binary variables, categorical variables and numerical variables
binary_columns = ["HadStroke","PneumoVaxEver","HadArthritis","HadKidneyDisease","DeafOrHardOfHearing","HadHeartAttack", "HadAngina", "ChestScan","DifficultyWalking","HadDiabetes","HadCOPD"]  # 已知的二元变量
categorical_columns = ["GeneralHealth", "SmokerStatus", "AgeCategory", "RemovedTeeth"]  # 分类变量
numerical_columns = ["PhysicalHealthDays", "MentalHealthDays", "BMI", "HeightInMeters", "WeightInKilograms"]  # 数值型变量

# Load data
df = pd.DataFrame()

# Numeric variable standardization
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# One-Hot Coding of Categorical Variables
# Assume the possible values of the categorical variable during training (obtained from the data set)
general_health_values = ["Poor", "Fair", "Good", "VeryGood", "Excellent"]
smoker_status_values = ["Current smoker - now smokes every day", "Current smoker - now smokes some days", "Former smoker", "Never smoked"]
age_category_values = ["Age 80 or older", "Age 75 to 79", "Age 70 to 74", "Age 65 to 69", "Age 60 to 64", "Age 55 to 59", "Age 50 to 54", "Age 40 to 44", "Age 35 to 39","Age 30 to 34","Age 25 to 29","Age 18 to 24",]
removed_teeth_values = ["1 to 5", "6 or more, but not all", "None of them", "All"]

# One-Hot Encoding
data = pd.get_dummies(data, columns=categorical_columns)

# Update categorical variable column names
# Get all newly generated column names
new_categorical_columns = [col for col in data.columns if any(cat_col in col for cat_col in categorical_columns)]

# Binary variable processing
def binary_encoder(x):
    return 1 if x == 'Yes' or x == 'Male' else 0

for col in binary_columns:
    data[col] = data[col].apply(binary_encoder)

# Merge all columns that need to be retained
columns_to_keep = binary_columns + new_categorical_columns + numerical_columns
df = data[columns_to_keep]
# Check out the final features
print("Final number of features：", df.shape[1])
print(df.head())

Final number of features： 42
   HadStroke  PneumoVaxEver  HadArthritis  HadKidneyDisease  \
0          0              0             0                 0   
1          0              0             0                 0   
2          0              0             0                 0   
3          0              1             1                 0   
4          0              1             0                 0   

   DeafOrHardOfHearing  HadHeartAttack  HadAngina  ChestScan  \
0                    0               0          0          0   
1                    0               0          0          0   
2                    0               0          0          0   
3                    0               0          0          1   
4                    0               0          0          1   

   DifficultyWalking  HadDiabetes  ...  AgeCategory_Age 80 or older  \
0                  0            1  ...                         True   
1                  0            0  ...                         Tr

In [4]:
# Separate training set, validation set and test set.
# Randomly partition the data using an 8:1:1 ratio.
from sklearn.model_selection import train_test_split

X = df.drop(['HadHeartAttack'], axis=1)
y = df['HadHeartAttack']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [5]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from joblib import delayed, Parallel

# Rebalance data. Over-sampling Category 1: Use methods such as SMOTE to increase the number of samples in Category 1.
from imblearn.over_sampling import SMOTE

# Upsampling using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Using an algorithm that penalizes imbalance In logistic regression, adjust class_weight="balanced" so that the model gives a higher weight to class 1.
# Define model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [6]:
# random forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
import joblib
feature_columns = X_train.columns.tolist()
# 假设训练好的模型是 log_reg
joblib.dump(rf_model, "rf_42features_model.pkl")  # 保存为 .pkl 文件
print("The model has been saved as rf_42features_model.pkl")
joblib.dump(feature_columns, "rf_42features_columns.pkl")

The model has been saved as rf_42features_model.pkl


['rf_42features_columns.pkl']

In [8]:
# Support vector machines are suitable for situations where the feature dimension is moderate and the data distribution is nonlinear.
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

# Define SVM model
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train, y_train)


SVC(probability=True, random_state=42)

In [9]:
# XGBoost is one of the implementations of gradient boosting trees and is suitable for complex nonlinear relationships between features.
from xgboost import XGBClassifier

# XGBoost model
xgb_model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, use_label_encoder=False)
xgb_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:32:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [10]:
# LightGBM is another efficient implementation of gradient boosted trees suitable for large-scale data.
import lightgbm as lgb
X_train.columns = X_train.columns.str.replace(r"[^\w]", "_", regex=True)  # 将非字母数字字符替换为下划线
X_val.columns = X_val.columns.str.replace(r"[^\w]", "_", regex=True)
# 定义 LightGBM 模型
lgb_model = lgb.LGBMClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 687, number of negative: 9438
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002147 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 626
[LightGBM] [Info] Number of data points in the train set: 10125, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.067852 -> initscore=-2.620165
[LightGBM] [Info] Start training from score -2.620165
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMClassifier(max_depth=6, random_state=42)

In [11]:
# Try logistic regression with regularization (such as L1 or L2 regularization).
from sklearn.linear_model import LogisticRegression

# Define logistic regression model (with regularization)
lr_model = LogisticRegression(penalty='l2', solver='liblinear', random_state=42)
lr_model.fit(X_train, y_train)


LogisticRegression(random_state=42, solver='liblinear')

In [12]:
# Load model
rf_model = joblib.load("rf_42features_model.pkl")
feature_columns = joblib.load("rf_42features_columns.pkl")

# Load user input sample data (please obtain user input according to the actual situation)
user_input = {
    "PhysicalHealthDays":5,
    "MentalHealthDays": 10,
    "BMI": 23,
    "HeightInMeters": 1.55,
    "WeightInKilograms": 70.68,
    "GeneralHealth": "Poor",       # 示例分类变量
    "SmokerStatus": "Yes",         # 示例分类变量
    "AgeCategory": "Age 30 to 34",   # 示例分类变量
    "RemovedTeeth": "None of them",   # 示例分类变量
    "HadStroke": "Yes",
    "PneumoVaxEver": "No",
    "HadArthritis": "No",
    "HadKidneyDisease": "Yes",
    "DeafOrHardOfHearing": "Yes",
    "HadHeartAttack": "No",
    "HadAngina": "Yes",
    "ChestScan": "No",
    "DifficultyWalking": "Yes",
    "HadDiabetes": "Yes",
    "HadCOPD": "Yes"
}
# Data preprocessing function
def preprocess_input(user_input):
    user_input_scaled = scaler.fit_transform([[user_input[col] for col in numerical_columns]])[0]
    numerical_input = dict(zip(numerical_columns, user_input_scaled))

    one_hot_encoded_input = {}
    for value in general_health_values:
        one_hot_encoded_input[f"GeneralHealth_{value}"] = 1 if user_input["GeneralHealth"] == value else 0
    for value in smoker_status_values:
        one_hot_encoded_input[f"SmokerStatus_{value}"] = 1 if user_input["SmokerStatus"] == value else 0
    for value in age_category_values:
        one_hot_encoded_input[f"AgeCategory_{value}"] = 1 if user_input["AgeCategory"] == value else 0
    for value in removed_teeth_values:
        one_hot_encoded_input[f"RemovedTeeth_{value}"] = 1 if user_input["RemovedTeeth"] == value else 0

    def binary_encoder(value):
        return 1 if value == "Yes" else 0

    binary_encoded_input = {col: binary_encoder(user_input[col]) for col in binary_columns}

    all_features = {**numerical_input, **one_hot_encoded_input, **binary_encoded_input}
    input_features = [all_features[col] if col in all_features else 0 for col in feature_columns]
    return np.array(input_features).reshape(1, -1)

# Convert to model input format (make sure the column order is consistent with training)
input_features = input_features = preprocess_input(user_input)

# Convert to NumPy array and predict
input_features = np.array(input_features).reshape(1, -1)
risk_probability = rf_model.predict_proba(input_features)[:, 1][0]  # Probability of getting sick
risk_class = rf_model.predict(input_features)[0]  # Classification results

# Output prediction results
print("heart disease risk probability:", round(risk_probability * 100, 2), "%")
print("Prediction category:", "high risk" if risk_class == 1 else "low risk")


heart disease risk probability: 48.0 %
Prediction category: low risk


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [13]:
# prediction function
def predict_risk(user_input):
    input_features = preprocess_input(user_input)
    risk_probability = rf_model.predict_proba(input_features)[:, 1][0]  # 得病概率
    risk_class = rf_model.predict(input_features)[0]  # 分类结果
    return round(risk_probability * 100, 2), "high risk" if risk_class == 1 else "low risk"
# Intervention simulation
interventions = {
    "Quit smoking": lambda input_data: {**input_data, "SmokerStatus": "No"},
    "Lose weight": lambda input_data: {**input_data, "BMI": max(18.5, input_data["BMI"] - 5)},
    "Improve mental health": lambda input_data: {**input_data, "MentalHealthDays": max(0, input_data["MentalHealthDays"] - 5)},
    "Reduce physical health problems": lambda input_data: {**input_data, "PhysicalHealthDays": max(0, input_data["PhysicalHealthDays"] - 5)},
    "Increase physical activity": lambda input_data: {**input_data, "DifficultyWalking": "No"},
    "Improve general health": lambda input_data: {**input_data, "GeneralHealth": "Excellent"},
    "Improve diet and oral health": lambda input_data: {**input_data, "RemovedTeeth": "None of them"},
    "Manage diabetes effectively": lambda input_data: {**input_data, "HadDiabetes": "No"},
    "Reduce heart-related risk factors": lambda input_data: {
        **input_data,
        "HadStroke": "No",
        "HadHeartAttack": "No",
        "HadAngina": "No",
        "ChestScan": "No",
    },
    "Improve respiratory health": lambda input_data: {**input_data, "HadCOPD": "No"},
    "Enhance kidney health": lambda input_data: {**input_data, "HadKidneyDisease": "No"},
    "Address arthritis issues": lambda input_data: {**input_data, "HadArthritis": "No"},
    "Assist hearing health": lambda input_data: {**input_data, "DeafOrHardOfHearing": "No"},
    "Encourage vaccination": lambda input_data: {**input_data, "PneumoVaxEver": "Yes"},
    "Encourage regular health check-ups": lambda input_data: {**input_data, "ChestScan": "No"},
}


# 初始风险
initial_probability, initial_class = predict_risk(user_input)
print("Initial heart disease risk probability:", initial_probability, "%")
print("Initial prediction category:", initial_class)

# 应用干预措施并预测效果
for name, intervention in interventions.items():
    modified_input = intervention(user_input)
    new_probability, new_class = predict_risk(modified_input)
    print(f"\nInterventions: {name}")
    print("Probability of heart disease risk after intervention:", new_probability, "%")
    print(":", new_class)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X

Initial heart disease risk probability: 48.0 %
Initial prediction category: low risk

Interventions: Quit smoking
Probability of heart disease risk after intervention: 48.0 %
: low risk

Interventions: Lose weight
Probability of heart disease risk after intervention: 48.0 %
: low risk

Interventions: Improve mental health
Probability of heart disease risk after intervention: 48.0 %
: low risk

Interventions: Reduce physical health problems
Probability of heart disease risk after intervention: 48.0 %
: low risk


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X


Interventions: Increase physical activity
Probability of heart disease risk after intervention: 44.0 %
: low risk

Interventions: Improve general health
Probability of heart disease risk after intervention: 47.0 %
: low risk

Interventions: Improve diet and oral health
Probability of heart disease risk after intervention: 48.0 %
: low risk

Interventions: Manage diabetes effectively
Probability of heart disease risk after intervention: 40.0 %
: low risk

Interventions: Reduce heart-related risk factors
Probability of heart disease risk after intervention: 13.0 %
: low risk

Interventions: Improve respiratory health
Probability of heart disease risk after intervention: 45.0 %
: low risk


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X


Interventions: Enhance kidney health
Probability of heart disease risk after intervention: 44.0 %
: low risk

Interventions: Address arthritis issues
Probability of heart disease risk after intervention: 48.0 %
: low risk

Interventions: Assist hearing health
Probability of heart disease risk after intervention: 45.0 %
: low risk

Interventions: Encourage vaccination
Probability of heart disease risk after intervention: 48.0 %
: low risk

Interventions: Encourage regular health check-ups
Probability of heart disease risk after intervention: 48.0 %
: low risk


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
